# Projet TSUNAMI : Topological EEG-Based Functional Connectivity Analysis for Motor Imagery
### Date : 2023-2024
*Auteurs : Diane CHOUNLAMOUNTRY & Orane VERNET*

*Encadrants : Mira RIZKALLAH, Aurélien VAN LANGHENHOVE*

# Introduction 
Dans ce notebook, on peut trouver toutes les fonctions que nous avons implémentées et les différentes méthodes que nous avons abordées afin de résoudre ce projet. Notons que nous n'avons pas forcément réussi à les rendre parfaitement discriminantes (distinction entre la main droite et la main gauche). Cependant, nous avons trouvé intéressant et utile de tout de même laisser ces méthodes, puisqu'elles pourront être reprises et améliorées par quiconque désirant poursuivre ce projet. Tous nos commentaires et remarques sont présents dans ce notebook afin de rendre nos résultats clairs.

# Etape 1 : importations des librairies

In [ ]:
#!pip install giotto-tda
#!pip install mne
#!pip install pandas
#!pip install networkx
#!pip install nxviz
#!pip install community
#!pip install gudhi
#!pip install persim

In [ ]:
#import the libraries and packages
import math
import os
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from nxviz import CircosPlot
import community
import gudhi
from gudhi import representations
import persim
import seaborn as sns
import itertools
import sys
from gtda.diagrams import PersistenceLandscape
import gtda.homology as hl

## Pour le machine learning
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


# Etape 2 : fonctions 
## Implémentation des différentes fonctions nécessaires

In [ ]:
def extract_trials(data):
    '''
    Fonction permettant d'extraire des signaux EEG raw les epochs et les labels (1:droite ; 2: gauche) de chaque événement
    '''
    # Charger les annotations
    annotations = data.annotations

    # Extraire les événements à partir des annotations
    events, event_id = mne.events_from_annotations(data, event_id={'769':1 , '770':2} ) #1=gauche et 2=droite

    epochs = mne.Epochs(data, events, event_id, tmin=1, tmax=5,baseline=None) #l'essai entier dure 8sec, on ne s'intéresse qu'à ce moment (c'est modifiable)

    label = events[:,2]
    
    return epochs.get_data(), label


In [ ]:
def matrice_de_correlation(essai):
    '''
    Prend en argument un essai et renvoie la matrice de correlation 
    de taille 32x32
    '''
    matrix = np.corrcoef(essai)

    return(matrix)

In [ ]:
def correlation_to_distance(matrice_corr, distance):
    '''
    Convertit une matrice de corrélation en une matrice de distance (distance choisie)
    '''
    if distance == '1':
        return (1-np.absolute(matrice_corr))
    if distance == '2':
        return (np.sqrt(-np.log(matrice_corr)))

In [ ]:
def persistence_barcode(matrice) : 
    '''
    En entrée, prend en argument la matrice de distance
    En sortie, plot le barcode correspondant
    '''

    rips_complex = gudhi.RipsComplex(distance_matrix=matrice, max_edge_length=1)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=3)
    diag = simplex_tree.persistence()
    
    gudhi.plot_persistence_barcode(diag, legend=True, max_intervals=0)

    plt.show()


In [ ]:
def persistence_diagram(matrice):
    '''
    max_edge_length = longueur maximale d'une arête pour être inclue dans le complexe de Rips
    max_dimension = dimension maximale des simplexes à inclure dans l'arbre
    diag est une liste de longueur 42 dont chaque élément est un tuple (classe topologique : 0, 1 ou 2 ; (birth, death))

    retourne le diagramme de persistence
    '''
    rips_complex = gudhi.RipsComplex(distance_matrix=matrice, max_edge_length=1)
    simplex_tree = rips_complex.create_simplex_tree(max_dimension=3)
    diag = simplex_tree.persistence()

    # Retourner le diagramme de persistence
    return diag

In [ ]:
def plot_persistence_diag(diag):
    '''
    Affichage d'un diagramme de persistence
    '''
    gudhi.plot_persistence_diagram(diag, legend=True, max_intervals=0)
    plt.tick_params(axis='both', labelsize=15)
    plt.xlabel('Birth', fontsize=15)
    plt.ylabel('Death', fontsize=15)
    plt.show()

In [ ]:
def reverse_diag(diag):
    '''
    Modifie l'ordre des éléments du tuple diag.
    La classe topologique est mise à la fin.
    (birth, death, classe topologique)
    '''
    reversed_diag = [(tpl[1][0], tpl[1][1], tpl[0]) for tpl in diag]
    return reversed_diag

In [ ]:
def landscape(diag,bins=200):
    '''
    Calcule et affiche le landscape
    '''
    reversed_diag = reverse_diag(diag)
    diag_array = np.array(reversed_diag)
    diag_array = np.expand_dims(diag_array, axis=0)  # Add a new axis

    landscape = PersistenceLandscape(n_layers=1, n_bins=bins).fit_transform_plot(diag_array)

    return landscape

In [ ]:
def data_to_landscape(data, num_essai, distance):
    '''
    Fonction qui récapitule la succession des fonctions précédentes
    Prend en entrée un signal entier (pas encore d'essais séparés) et qui
    renvoie le landscape de l'essai précisé en argument.
    '''
    
    epochs, label = extract_trials(data)
    essai=epochs[num_essai,:,:]
    matrice_correlation = matrice_de_correlation(essai)
    matrice_distance = correlation_to_distance(matrice_correlation, distance)
    #persistence_barcode(matrice_distance)
    diag = persistence_diagram(matrice_distance)
    landscape_result = landscape(diag,bins=200)

    return landscape_result, reverse_diag(diag), matrice_correlation

In [ ]:
def creation_liste_BO(personne, distance):
    '''
    Il faut juste choisir la personne sur laquelle on travaille et le type de distance
    personne = 'A1', ... 'A60'
    distance = '1' ou '2'
    '''
    
    data_path1 = IMDIR + '/' + personne + '/' + personne + '_R1_acquisition.gdf'
    data_path2 = IMDIR + '/' + personne + '/' + personne + '_R2_acquisition.gdf'
    
    raw1 = mne.io.read_raw_gdf(data_path1, preload=True)
    data1=raw1.copy()
    
    raw2 = mne.io.read_raw_gdf(data_path2, preload=True)
    data2=raw2.copy()
    
    epochs1, label1 = extract_trials(raw1)
    epochs2, label2 = extract_trials(raw2)
    
    label1 = label1.tolist()
    label2 = label2.tolist()
    
    labels = label1 + label2
    
    # Liste qui contient les landscapes des 80 essais en boucles ouvertes
    landscapes_BO = []
    diag_BO = []
    matrice_corr_BO = []
    
    for num_essai in range (40):
        l, diag, matrice_corr= data_to_landscape(data1, num_essai, distance)
        landscapes_BO.append(l)
        diag_BO.append(diag)
        matrice_corr_BO.append(matrice_corr)
    for num_essai in range(40):
        l, diag, matrice_corr = data_to_landscape(data2, num_essai, distance)
        landscapes_BO.append(l)
        diag_BO.append(diag)
        matrice_corr_BO.append(matrice_corr)


    return landscapes_BO, diag_BO, matrice_corr_BO, labels


In [ ]:
def creation_liste_BF(personne, distance):
    '''
    Il faut juste choisir la personne sur laquelle on travaille et le type de distance
    personne = 'A1', ... 'A60'
    distance = '1' ou '2'
    C'est une liste des matrices de corrélation pour les essais effectués en boucle fermée
    Ce sont nos données test pour le machine learning
    '''
    
    data_path3 = IMDIR + '/' + personne + '/' + personne + '_R3_onlineT.gdf'
    data_path4 = IMDIR + '/' + personne + '/' + personne + '_R4_onlineT.gdf'
    data_path5 = IMDIR + '/' + personne + '/' + personne + '_R5_onlineT.gdf'
    data_path6 = IMDIR + '/' + personne + '/' + personne + '_R6_onlineT.gdf'

    data_paths = [data_path3, data_path4, data_path5, data_path6]
    datas = []
    labels = []

    for i in range(4):
        raw = mne.io.read_raw_gdf(data_paths[i], preload=True)
        data = raw.copy()
        epochs, label = extract_trials(raw)
        label = label.tolist()
        datas.append(data)
        labels.extend(label)


    # Liste qui contient les landscapes des 40*4=160 essais en boucles fermées
    landscapes_BF = []
    diag_BF = []
    matrice_corr_BF = []

    for num_essai in range(len(labels)):   
        num_essai2 = num_essai//40
        l, diag, matrice_corr= data_to_landscape(datas[num_essai2], num_essai2, distance) #modifier datas en faisant *40
        landscapes_BF.append(l)
        diag_BF.append(diag)
        matrice_corr_BF.append(matrice_corr)

    
    return landscapes_BF, diag_BF, matrice_corr_BF, labels

In [ ]:
def liste_to_matrice(liste_matrices_correlation):
    '''
    Convertir la liste des matrices de correlation en un tableau numpy
    Chaque ligne est une matrice applatie (juste sa supérieure, car symétrie)

    ici ce sera pour : liste_matrice = matrice_corr_80_BO
    '''
    tab_des_matrices = []
    for matrice in liste_matrices_correlation : 
        matrice_list = matrice[np.triu_indices(matrice.shape[0])]
        tab_des_matrices.append(matrice_list)

    res = np.array(tab_des_matrices)

    return res

In [ ]:
def beti_all(liste_diag):
    '''
    Enlève le troisième élément du tuple (0, 1 et 2) : ne reste que (birth, death) pour tous les beti numbers
    '''

    liste_beti = []
    for diag in liste_diag:
        diag_list = []
        for elt in diag :
            bd = (elt[0], elt[1])
            diag_list.append(bd)
        liste_beti.append(diag_list)
        
    return liste_beti

In [ ]:
def find_best_accuracy(kmin, kmax, nb_tirages, matrices_corr, labels, metric = 'minkowski'):
    '''
    J'effectue plusieurs tirages, c'est-à-dire plusieurs séparation train/test différentes.
    Pour chacun de ces tirages, j'observe la précision de l'algorithme knn en fonction de la valeur de k (nombre de voisins)
    Dans "matrice_accuracy", je stocke sur chaque ligne la précision du tirages pour plusieurs k (nb_lignes = nb_tirages ; nb_colonnes = nb_K)
    Je renvoie ensuite la moyenne et l'écart-type des précisions pour chaque nombre de voisins k.
    -----------------
    La métrique par défaut est celle de 'minkowski', elle est modifiable.
    '''

    acc_mean = []
    acc_std = []
    
    for k in range (kmin, kmax+1):
        accuracy_k = []
        for tirages in range(nb_tirages):
            X_train, X_test, y_train, y_test = train_test_split(matrices_corr, labels, test_size=0.3)
            knn = KNeighborsClassifier(n_neighbors=k, metric = metric)
            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)

            accuracy = accuracy_score(y_test, y_pred)
            accuracy_k.append(accuracy)

        accuracy_k_array = np.array(accuracy_k)
        acc_mean.append(np.mean(accuracy_k_array))
        acc_std.append(np.std(accuracy_k_array))
    
    return acc_mean, acc_std

In [ ]:
def accuracy_beti_wassertein(kmin, kmax, nb_tirages, diag, labels):
    '''
    -------------------------------------------------------
    Métrique utilisée : distance de wasserstein
    Algorithme de ML : KNN
    ---------------------------------------------------------
    J'effectue plusieurs tirages, c'est-à-dire plusieurs séparation train/test différentes.
    Pour chacun de ces tirages, j'observe la précision de l'algorithme knn en fonction de la valeur de k (nombre de voisins)
    Dans "matrice_accuracy", je stocke sur chaque ligne la précision du tirages pour plusieurs k (nb_lignes = nb_tirages ; nb_colonnes = nb_K)
    Je renvoie ensuite la moyenne et l'écart-type des précisions pour chaque nombre de voisins k.
    
    '''
    liste_beti = beti_all(diag)
    
    # Cette matrice augmente d'une ligne à chaque tirage
    # nb_lignes = nb_tirages ; nb_colonnes = nb_K
    matrice_accuracy = np.array([]).reshape(0, kmax-kmin+1)
    
    for tirages in range(nb_tirages):
        beti_train, beti_test, labels_train, labels_test = train_test_split(liste_beti, labels, stratify=labels, test_size=0.3)
        
        # Calculs des matrices de distance
        wasser_dist_train = gudhi.representations.metrics.WassersteinDistance()
        wasser_dist_train.fit(X=beti_train)
        dist_matrix_wasser_train = wasser_dist_train.transform(beti_train)
          
        wasser_dist_test = gudhi.representations.metrics.WassersteinDistance()
        wasser_dist_test.fit(X=beti_test)
        dist_matrix_wasser_test = wasser_dist_test.transform(beti_train)
        
        accuracy_k= []
        for k in range (kmin, kmax+1):  
            
            # Algo KNN
            knn_classifier = KNeighborsClassifier(n_neighbors=k, metric='precomputed')
            knn_classifier.fit(dist_matrix_wasser_train, labels_train)
            predictions = knn_classifier.predict(dist_matrix_wasser_test.T)
                
             # Calculer la précision
            accuracy = accuracy_score(labels_test, predictions)
            accuracy_k.append(accuracy)
            
        accuracy_k_array = np.array(accuracy_k)

        matrice_accuracy = np.vstack([matrice_accuracy, accuracy_k_array])

    # calcul de la moyenne et de l'écart-type
    acc_mean = np.mean(matrice, axis=0)
    acc_std = np.std(matrice, axis=0)

    return acc_mean, acc_std

In [ ]:
def separation_dg(landscapes_list, labels):
    '''
    En entrée : les landscapes à étudier en liste et les labels (1/2) correspondants
    En sortie : renvoie les liste des beti_i pour les essais main gauche et main droite séparés
    '''

    ## 1. Récuper le numéro des landscapes main droite "1"
    
    num_main_droite = [i for i in range(len(labels)) if labels[i] == 1]
    num_main_gauche = [i for i in range(len(labels)) if labels[i] == 2]
    
    ## 2. Séparer les landscapes en deux listes
    
    landscapes_main_droite = [landscapes_list[i] for i in num_main_droite]
    landscapes_main_gauche = [landscapes_list[i] for i in num_main_gauche]
    
    ## 3. Séparer les beti1 et les beti2
    
    beti1_droite= [landscapes_main_droite[land][0][1] for land in range(len(landscapes_main_droite))]
    beti1_gauche= [landscapes_main_gauche[land][0][1] for land in range(len(landscapes_main_gauche))]
    
    #On fait pareil pour les beti2, mais attention : pour les landscapes qui sont de dimension 3
    beti2_droite= [landscapes_main_droite[land][0][2] for land in range(len(landscapes_main_droite)) if landscapes_main_droite[land].shape[1]==3]
    beti2_gauche= [landscapes_main_gauche[land][0][2] for land in range(len(landscapes_main_gauche)) if landscapes_main_gauche[land].shape[1]==3]


    return beti1_droite, beti1_gauche, beti2_droite, beti2_gauche



In [ ]:
def moyenne_beti_numbers(beti1_droite, beti1_gauche, beti2_droite, beti2_gauche):
    '''
    En entrée : les données des beti numbers 1 et 2 pour la main droite et la main gauche
    En sortie : - calcule la moyenne pour chaque catégorie  
  
    '''

    ## 4. Calcul de la moyenne
    
    #pour beti1
    
    beti1_droite_array = np.array(beti1_droite)
    moy_beti1_droite = np.mean(beti1_droite_array, axis=0)  
    
    beti1_gauche_array = np.array(beti1_gauche)
    moy_beti1_gauche = np.mean(beti1_gauche_array, axis=0)
   
    
    # pour beti 2
    
    beti2_droite_array = np.array(beti2_droite)
    moy_beti2_droite = np.mean(beti2_droite_array, axis=0)
 
    beti2_gauche_array = np.array(beti2_gauche)
    moy_beti2_gauche = np.mean(beti2_gauche_array, axis=0)
    

    return moy_beti1_droite, moy_beti1_gauche, moy_beti2_droite, moy_beti2_gauche


In [ ]:
def var_beti_numbers(beti1_droite, beti1_gauche, beti2_droite, beti2_gauche):
    '''
    En entrée : les données des beti numbers 1 et 2 pour la main droite et la main gauche
    En sortie : - calcule la variance pour chaque catégorie  
                
    '''

    ## 5. Calcul de la variance
    
    #pour beti1

    beti1_droite_array = np.array(beti1_droite)
    var_beti1_droite = np.var(beti1_droite_array, axis=0)

    beti1_gauche_array = np.array(beti1_gauche)
    var_beti1_gauche = np.var(beti1_gauche_array, axis=0)
    
  
    # pour beti 2
    
    beti2_droite_array = np.array(beti2_droite)
    var_beti2_droite = np.var(beti2_droite_array, axis=0)

    beti2_gauche_array = np.array(beti2_gauche)
    var_beti2_gauche = np.var(beti2_gauche_array, axis=0)

    return var_beti1_droite, var_beti1_gauche, var_beti2_droite, var_beti2_gauche
    

In [ ]:
def std_beti_numbers(beti1_droite, beti1_gauche, beti2_droite, beti2_gauche):
    '''
    En entrée : les données des beti numbers 1 et 2 pour la main droite et la main gauche
    En sortie : - calcule l'écart-type pour chaque catégorie  
                
    '''

    var_beti1_droite, var_beti1_gauche, var_beti2_droite, var_beti2_gauche = var_beti_numbers(beti1_droite, beti1_gauche, beti2_droite, beti2_gauche)

    # on prend la racine carrée élément par élément pour les 4 vecteurs

    std_beti1_droite = np.sqrt(var_beti1_droite)
    std_beti1_gauche = np.sqrt(var_beti1_gauche)
    std_beti2_droite = np.sqrt(var_beti2_droite)
    std_beti2_gauche = np.sqrt(var_beti2_gauche)

    beti1_droite_array = np.array(beti1_droite)
    moy_beti1_droite = np.mean(beti1_droite_array, axis=0)

    return std_beti1_droite, std_beti1_gauche, std_beti2_droite, std_beti2_gauche

In [ ]:
def plot_mean_with_std(mean, std, title, xlabel, ylabel):
    '''
    Fonction d'affichage prenant entrée une liste de moyennes et une liste d'écart-type.
    Affichage de la moyenne en ligne pleine
    Lignes pointillées : moyenne + écart-type et moyenne - écart-type
    '''
    
    x = np.arange(len(mean))  
    
    plt.plot(x, mean, label='Moyenne')  # Courbe de la moyenne
    
    # Courbe pour l'écart-type au-dessus
    plt.plot(x, mean + std, linestyle='--', label='Moyenne + Écart-type')
    
    # Courbe pour l'écart-type en dessous
    plt.plot(x, mean - std, linestyle='--', label='Moyenne - Écart-type')
    
    plt.fill_between(x, mean - std, mean + std, color='gray', alpha=0.2)  # Remplissage entre les courbes d'écart-type
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.grid(True)

# Etape 3 : Récupération des données 
**Remarques :**
* Par souci de simplicité, nous avons travaillé uniquement avec la base de donnée A.
* Il faut choisir le sujet sur lequel on élabore le modèle.
* Attention : ne pas entraîner et tester un modèle avec plusieurs sujets différents ! En effet, les séries temporelles ne sont pas stationnaires c'est-à-dire que la moyenne et la variance ne sont pas constantes. On ne peut pas utiliser un même modèle pour plusieurs personnes différentes. L'objectif est plutôt de trouver une méthode efficace sur un sujet et voir si elle l'est aussi avec les autres.

In [ ]:
# Chemin vers les données
# On se concentre sur les sujets de la base de données A
IMDIR = '../BCI_Database/BCI_Database/Signals/DATA_A'

In [ ]:
# On choisit la personne sur laquelle on travaille
personne = 'A19'
distance = '1'

# BO (2 calibration runs - 80 essais)
landscapes_BO, diag_BO, matrice_corr_BO, labels_BO = creation_liste_BO(personne = personne, distance = distance)

# BF (4 runs - 160 essais)
landscapes_BF, diag_BF, matrice_corr_BF, labels_BF = creation_liste_BF(personne = personne, distance = distance)

# Concaténation des BO et des BF
diag_all = diag_BO + diag_BF
labels_all = labels_BO + labels_BF
landscapes_all = landscapes_BO + landscapes_BF
matrice_corr_all = matrice_corr_BO + matrice_corr_BF

# Etape 4 : Analyse des données / Machine Learning
## Objectif général : discriminer une intention de mouvement de la main droite ou de la main gauche

**Remarques introductives :**
* Dans ce projet, nous avons utilisé algorithme de K-NN (K-Nearest Neighbors) comme algorithme de Machine Learning. Notons que nous avons dédié notre temps de fin de projet à essayer d'améliorer la précision de KNN pour trouver où étaient nos possibles erreurs d'implémentations et d'hypothèses. De ce fait, nous n'avons pas testé d'autres algorithmes de Machine Learning ; c'est quelque chose qui serait justement intéressant à essayer dans de futures continuations.
* Dans la suite, nous avons exposé les trois différentes pistes de discrimination que nous avons explorées.

## 1. Etude des landscapes

**Objectif :** Utiliser la moyenne des nombres de Beti et leur écart-type et réussir à discerner un changement entre les landscapes provenant des essais main droite et ceux des essais main gauche.

**Résultats :** Chaque landscape a un axe des abscisses qui lui est propre. Donc lorsqu'on calcule la moyenne des nombres de Beti, les valeurs numériques sont pas correspondantes entre plusieurs landscapes --> ce qu'on obtient n'est pas interprétable. Nous avons décidé de ne pas appliquer d'algorithme de Machine Learning à ces données.

**Suite :** Il faudrait créer un axe d'abscisse généralisé à tous les landscapes. Nous avons choisi de ne pas développer cette piste mais de plutôt trouver une représentation plus généralisée.

In [ ]:
beti1_droite, beti1_gauche, beti2_droite, beti2_gauche = separation_dg(landscapes_BO, labels_BO)
moy_beti1_droite, moy_beti1_gauche, moy_beti2_droite, moy_beti2_gauche = moyenne_beti_numbers(beti1_droite, beti1_gauche, beti2_droite, beti2_gauche)
std_beti1_droite, std_beti1_gauche, std_beti2_droite, std_beti2_gauche = std_beti_numbers(beti1_droite, beti1_gauche, beti2_droite, beti2_gauche)



plt.figure(figsize=(12,8))
plt.subplots_adjust(hspace=0.3)
plt.subplots_adjust(wspace=0.2)

#Beti 1
plt.subplot(221)
plot_mean_with_std(moy_beti1_gauche,std_beti1_gauche, 'Beti1 - Main gauche','x','y')
plt.subplot(222)
plot_mean_with_std(moy_beti1_droite,std_beti1_droite, 'Beti1 - Main droite','x','y')

#Beti 2
plt.subplot(223)
plot_mean_with_std(moy_beti2_gauche,std_beti2_gauche, 'Beti2 - Main gauche','x','y')
plt.subplot(224)
plot_mean_with_std(moy_beti2_droite,std_beti2_droite, 'Beti2 - Main droite','x','y')


## 2. Discriminer les matrices de corrélation

**Objectifs :** Trouver une représentation simple et généralisée pour chaque essai (contrairement au landscape). Pourrait-elle réussir à distinguer les deux intentions de mouvement différents ?

**Résultats :** La précision du modèle n'est pas satisfaisante.

**Suite :** Trouver des types de distance plus appropriés pour nos données.

In [ ]:
kmin = 1
kmax = 30
nb_tirages = 20
matrices_corr = liste_to_matrice(matrice_corr_all)
labels = labels_all

acc_mean1, acc_std1 = find_best_accuracy(kmin, kmax, nb_tirages, matrices_corr, labels, metric = 'cityblock')
acc_mean2, acc_std2 = find_best_accuracy(kmin, kmax, nb_tirages, matrices_corr, labels, metric = 'euclidean')
acc_mean3, acc_std3 = find_best_accuracy(kmin, kmax, nb_tirages, matrices_corr, labels, metric = 'nan_euclidean') 
acc_mean4, acc_std4 = find_best_accuracy(kmin, kmax, nb_tirages, matrices_corr, labels, metric = 'l1') 

In [ ]:
# Affichage

title = 'Précision du modèle selon K, moyennée sur ' + str(nb_tirages) + ' tirages \n metric = '
xlabel = 'K : nombre de voisins'
ylabel = 'Précision'

plt.figure(figsize=(14,8))
plt.subplots_adjust(hspace=0.4)
plt.subplots_adjust(wspace=0.2)

plt.subplot(221)
plot_mean_with_std(np.array(acc_mean1), np.array(acc_std1), title + 'cityblock', xlabel, ylabel)

plt.subplot(222)
plot_mean_with_std(np.array(acc_mean2), np.array(acc_std2), title + 'euclidean' , xlabel, ylabel)

plt.subplot(223)
plot_mean_with_std(np.array(acc_mean1), np.array(acc_std1), title + 'nan_euclidean', xlabel, ylabel)

plt.subplot(224)
plot_mean_with_std(np.array(acc_mean2), np.array(acc_std2), title + 'l1' , xlabel, ylabel)

## 3. Des matrices de distance plus appropriées

**Objectifs :** Donner à l'algorithme de KNN des matrices de distance pré-calculées pour tenter des meilleures précisions

**Résultats :** Les précisions restent assez peu satisfaisantes. On a testé en utilisant que les essais en BO (80 essais) ou que en BF (140 essais). Nous avons essayé avec la concaténation B0+BF par pour une raison qu'on ignore, le programme prend beaucoup plus de temps à tourner (même en réduisant le nombre de tirages, nous n'arrivons pas à obtenir un graphique). Les résultats sont similaires entre les deux types d'essais.

**Suite :** Changer d'autres paramètres pour arriver à de meilleurs résultats.

In [ ]:
kmin = 1
kmax = 25

acc_mean, acc_std = accuracy_beti_wassertein(kmin, kmax, nb_tirages=20, diag = diag_BO, labels = labels_BO)
acc_mean2, acc_std2 = accuracy_beti_wassertein(kmin, kmax, nb_tirages=5, diag = diag_BF, labels = labels_BF)


In [ ]:
# Tracer l'évolution de la précision selon le nombre de voisins
plt.figure(figsize=(12,4))

plt.subplot(121)
plot_mean_with_std(acc_mean, acc_std, 'Précision du modèle selon K, moyennée sur ' + str(20) + ' tirages \n Distance de Wasserstein - BO uniquement', xlabel, ylabel)

plt.subplot(122)
plot_mean_with_std(acc_mean2, acc_std2, 'Précision du modèle selon K, moyennée sur ' + str(5) + ' tirages \n Distance de Wasserstein - BF uniquement', xlabel, ylabel)

# Conclusion et remerciements
Nous concluons ce notebook ici et avec lui le projet TSUNAMI. Ce dernier nous a permis de travailler sur un sujet innovateur avec un objectif concret, ce qui l'a rendu très intéressant. Par ailleurs, il nous a fait découvrir l'analyse topologique, qui est un type de représentation que nous n'avions encore jamais eu l'occasion d'étudier. Ce fut une part non négligeable de notre projet que de nous approprier ces outils de représentation qui nous étaient inconnus. De plus, nous avons eu la chance d'être encadrées par un ingénieur travaillant au CHU et ayant ainsi un point de vue plus pragmatique sur le sujet que nous. En effet, dès le début du projet, Aurélien a bien insisté sur le contexte médical et sur le côté pratique des expériences. Nous avons trouvé cela très utile car nous sommes plus habituées à travailler sur de la théorie ; ainsi, il est important de ne pas oublier que les travaux fournis ont comme objectif de servir dans la vie réelle.

Enfin, ce projet nous a aussi introduit au monde de la recherche et aux frustrations qui viennent avec. En effet, nous ne pouvons déclarer avoir terminé ce projet puisqu'il sera toujours possible de le continuer et de l'améliorer. De plus, par manque de temps à la fin du projet, nous n'avons pas réussi à obtenir des résultats assez satisfaisants. Nous sommes néanmoins fières du travail réalisé et des efforts fournis.

Nos remerciements vont à Mira RIZKALLAH, à Aurélien LANGHENHOVE et à Maria SARKIS qui nous auront encadrées toujours dans la bonne humeur.

# Références bibliographiques
1. Dreyer P., Roc A., Pillette L., Rimbert S., Lotte F. (2023). *A large EEG database with users' profile information for motor imagery.*

2. Gervini Zampieri Centeno E., Moreni G., Vriend C., Douw L., Antônio Nobrega Santos F. (2022). *A hands-on tutorial on network and topological neuroscience*

3. Yan Y. et al., *"Topological EEG-Based Functional Connectivity Analysis for Mental Workload State Recognition"*, in IEEE Transactions on Instrumentation and Measurement, vol. 72, pp. 1-14, 2023.